In [25]:
# package imports
import numpy as np
from itertools import islice
from scipy.linalg import eig
from scipy.stats import norm

In [26]:
# get stationary dustribution of transition matrix
# from stack overflow
def get_stationary_distibution(state_transition_matrix: np.ndarray) -> np.ndarray:
    S, U = eig(state_transition_matrix.T)
    stationary = np.array(U[:, np.where(np.abs(S - 1.) < 1e-8)[0][0]].flat)
    stationary = stationary / np.sum(stationary)
    return stationary

In [27]:
def estimate_sequence(state_transition_matrix:np.ndarray, gaussian_params: np.ndarray, initial_state_probability:np.ndarray, observations:np.ndarray, state_count: int):
    observation_count = observations.shape[0]
    state_probability_matrix = np.zeros((state_count, observation_count))
    path = np.ndarray((state_count, observation_count-1), dtype=int)
    cond_prob = np.ndarray((state_count, observation_count))
    # cond_prob = norm.pdf(observations[0], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
    cond_prob[:,0] = norm.pdf(observations[0], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
    state_probability_matrix[:,0] = np.log2(initial_state_probability) + np.log2(cond_prob[:,0])
    for i in range(1, observation_count):
        # cond_prob = norm.pdf(observations[i], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
        cond_prob[:,i] = norm.pdf(observations[i], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
        for j in range(state_count):
            # prob = state_probability_matrix[:,i-1] + np.log2(transition_matrix[:,j]) + np.log2(cond_prob[j])
            prob = state_probability_matrix[:,i-1] + np.log2(state_transition_matrix[:,j]) + np.log2(cond_prob[j,i])
            path[j,i-1] = np.argmax(prob)
            state_probability_matrix[j,i] = max(prob)

    return path, state_probability_matrix, cond_prob
    # for i in range(1, observation_count):
    # cond_prob = norm.pdf(observations[1], loc=gaussian_params[:,0], scale=gaussian_params[:,1])
    # state_probability_matrix[0,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix[:,0]) + np.log2(cond_prob[0]))
    # state_probability_matrix[1,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix[:,1]) + np.log2(cond_prob[1]))
    # print(state_probability_matrix[:,1])
    # state_probability_matrix[:,1] = max(state_probability_matrix[:,0] + np.log2(transition_matrix) + np.log2(cond_prob))
    # print(state_probability_matrix[:,1])

In [28]:
# Viterbi algorithm
def viterbi():
    pass

In [29]:
# Baum-Welch Learning
def baum_welch():
    pass

In [30]:
# read data
observed_states = np.loadtxt('./Input/data.txt', dtype=float)
observed_states.shape

(1000,)

In [31]:
# read parameters
with open('./Input/parameters.txt.txt', 'r') as f:
    no_of_states = int(f.readline())

with open('./Input/parameters.txt.txt', 'r') as lines:
    transition_matrix = np.genfromtxt(islice(lines, 1, 1+no_of_states))

with open('./Input/parameters.txt.txt', 'r') as lines:
    gaussian_parameters = np.genfromtxt(islice(lines, 1+no_of_states, 1+2*no_of_states), dtype=int)

gaussian_parameters

array([[200, 100],
       [ 10,  10]])

In [32]:
initial_distribution = get_stationary_distibution(transition_matrix)
initial_distribution

array([0.25, 0.75])

In [33]:
a, b, c = estimate_sequence(transition_matrix, gaussian_parameters, initial_distribution, observed_states, no_of_states)

In [34]:
# norm.pdf(104.524317662043, loc=gaussian_parameters[:,0], scale=gaussian_parameters[:,1])
a

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [35]:
b

array([[  -10.62715623,   -19.85748309,   -29.04578103, ...,
        -8885.02327226, -8894.28602329, -8903.51009446],
       [  -69.51400828,   -73.24859934,   -86.24148327, ...,
        -9021.89941576, -8944.89420803, -8957.4530169 ]])

In [36]:
c

array([[2.52910697e-03, 2.37846467e-03, 2.44877380e-03, ...,
        3.50106825e-03, 2.32560557e-03, 2.38880035e-03],
       [1.58174739e-21, 4.69835681e-19, 3.46181183e-20, ...,
        5.10919498e-44, 3.16177680e-18, 3.21898484e-19]])